##### Step 1: Initialize Workspace & Environment

In [6]:
from azureml.core import Workspace, Model
from azureml.core import Environment

# Initialize workspace
ws = Workspace.from_config()

# Get the trained model from registry
model_name= 'pima_model_SDKv1_02'
model = ws.models[model_name]
print(model.name, 'version', model.version)


# environment name
env_name= 'ENV-SDKv1'

# ---- create enviroment using .yaml file
# python_packages = Environment.from_conda_specification(env_name, '../dependencies/conda.yaml')
# # register the environment
# python_packages.register(workspace=ws)


pima_model_SDKv1_01 version 5
pima_model_SDKv1_02 version 1


Step 3: Configure & Deploy Model as a Web Service

In [7]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment.get(workspace=ws, name=env_name)
service_env.inferencing_stack_version="latest" # to use latest version

inference_config = InferenceConfig(source_directory='../src/',
                                   entry_script= 'pima_scoreRealtimeEndpoint_SDKv1.py',
                                   environment=service_env
                                   )

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)

# Deploy the model as a service
print('Deploying model...')
service_name = "pima-indians-sdkv1-service3"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)
print(service.get_logs())

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-11-10 13:15:54+00:00 Creating Container Registry if not exists.
2023-11-10 13:15:55+00:00 Use the existing image.
2023-11-10 13:15:55+00:00 Generating deployment configuration.
2023-11-10 13:15:55+00:00 Submitting deployment to compute.
2023-11-10 13:15:59+00:00 Checking the status of deployment pima-indians-sdkv1-service3.

/tmp/ipykernel_48326/4153428433.py:20: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
